## SI 670 Applied Machine Learning, Week 3: Cross Validation, Logistic Regression and Evaluation (Due Wednesday Sept 21, 2022 11:59pm)

For this assignment, you will be exercising on questions related to Logistic regression, Dummy classifiers, and cross-validation.

* This homework is worth 100 points in total. Correct answers and code receive full credit, but partial credit will be awarded if you have the right idea even if your final answers aren't quite right.

* Submit your completed notebook file to the Canvas site - **IMPORTANT**: please name your submitted file `si670f22-hw3-youruniqname.ipynb`

* Any file submitted after the deadline will be marked as late. Please consult the syllabus regarding late submission policies. You can submit the homework as many time as you want, but only your latest submission will be graded.

* As a reminder, the notebook code you submit must be your own work. Feel free to discuss general approaches to the homework with classmates. If you end up forming more of a team discussion on multiple questions, please include the names of the people you worked with at the top of your notebook file.



### Question 1 (20 points)

This question asks you to explore the relationship between the distribution of cross-validation scores vs the data size. 

Please use a 'for' loop to repeat the following steps with `n` varying from `[100, 500, 1000, 5000, 10000]`:
1. Generate synthetic data `(X_F1, y_F1)` using `make_friedman1` with `n_samples=n, n_features=7, random_state=0`;
2. Create a `LinearRegression` model
3. Apply `cross_val_score` to the model and generated data and use three-fold cross-validation.
4. Calculate the mean and std of the returned `cv_scores` across the 3 cross-validation folds.

Store the mean and std of the `cv_scores` in lists `cv_scores_mean` and `cv_scores_std` respectively. Do you see the std increasing or decreasing? And why? 

*This function should return a tuple of two lists `(cv_scores_mean, cv_scores_std)`, where both of them should have length 5.*


In [ ]:
def answer_one():
    import numpy as np
    from sklearn.datasets import make_friedman1
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import cross_val_score
    
    cv_scores_mean = []
    cv_scores_std = []
    # Your code here
    
    return (cv_scores_mean, cv_scores_std)

# answer_one()

Now, you will train a logistic regression classifier models and evaluate how effectively it  predicts instances of fraud using data based on [this dataset from Kaggle](https://www.kaggle.com/dalpozz/creditcardfraud). Then you'll perform a grid search to find optimal parameters. 
 
Each row in `fraud_data.csv` corresponds to a credit card transaction. Features include confidential variables `V1` through `V28` as well as `Amount` which is the amount of the transaction. 
 
The target is stored in the `class` column, where a value of 1 corresponds to an instance of fraud and 0 corresponds to an instance of not fraud.

In [ ]:
# # run this cell if you are using Colab

# !rm -rf fraud_data.csv

# import io

# from google.colab import files as colab_files
# uploaded = colab_files.upload()
# files = {'fraud_data.csv': io.BytesIO(uploaded['fraud_data.csv'])}

# # upload 'fraud_data.csv'

In [ ]:
# run this cell if you are using Jupyter
files = {'fraud_data.csv': 'fraud_data.csv'}

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

df = pd.read_csv(files['fraud_data.csv'])

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Question 2 (20 points)

Using `X_train`, `X_test`, `y_train`, and `y_test` (as defined above), train a dummy classifier that classifies everything as the majority class of the training data. What is the accuracy of this classifier? What is the recall?

Then train a LogisticRegression classifier with C=1. What is the accuracy? What is the recall?

*This function should a return a tuple with four floats, i.e. `(dummy_accuracy, dummy_recall, lr_accuracy, lr_recall)`.*

In [ ]:
def answer_two():
    # YOUR CODE HERE
    
    return dummy_accuracy, dummy_recall, lr_accuracy, lr_recall

# answer_two()

### Question 3 (20 points)

Fit the LogisticRegression with `C` varying from `[[0.1, 1, 10]` and report the accuracy, precision, recall, and F1 scores for each choice of `C`.

*This function should a return a tuple with four lists, i.e. `(accuracy_list, precision_list, recall_list, f1_list)`, and each list should contain 3 numbers, one per choice of C*

In [ ]:
def answer_three():

    # YOUR CODE HERE

    return accuracy_list, precision_list, recall_list, f1_list

# answer_three()

### Question 4 (20 points)

Train a logistic regression classifier with `C=10` using X_train and y_train.

For the logistic regression classifier, create (1) a precision-recall curve and (2) an ROC curve using y_test and the probability estimates for X_test (probability it is fraud).

Looking at the precision recall curve, what is the recall when the precision is `0.75`?

Looking at the ROC curve, what is the true positive rate when the false positive rate is `0.16`?

*This function should return a tuple with two floats, i.e. `(recall_at_p75, tpr_at_fpr16)`.*
*You should also include code that uses the scikit-learn (sklearn.metrics) functions precision_recall_curve and roc_curve to generate the precision/recall and ROC curves above*

In [ ]:
def answer_four():

    # YOUR CODE HERE

    return recall_at_p75, tpr_at_fpr16

# answer_four()

### Question 5 (20 Points)

Suppose you have trained a classifier distinguishing Benign vs Malignant cancers. And the confusion matrix of your classifier is given below.

|      	| Predicted: Benign 	| Predicted: Malignant 	|
|:----:	|----	|----	|
| Actual: Benign 	|    10000	|   100 	|
| Actual: Malignant  	|    200	|   10 	|

### Question 5 (a) (10 points) 
If we assume Benign is the positive class and Malignant is the negative class, what are the precision and recall for Benign? If we assume Malignant is the positive class and Benign is the negative class, what are the precision and recall for Malignant?

*This function should return a tuple of four float numbers: `(precision_benign, recall_benign, precision_malignant, recall_malignant)`. You can calculate these scores either by coding, or by hand using the correct formulas for precision and recall.*

In [ ]:
def answer_five_a():
    
    # Coding or manually calculating the precision and recall scores here. 

    return precision_benign, recall_benign, precision_malignant, recall_malignant

# answer_five_a()

### Question 5 (b) (10 points) 
If you have another classifier with the following confusion matrix, which classifier do you prefer and why? (Hint: calculate the precision and recall scores for this classifier and compare with the previous classifier, considering what the relative costs are for each type of mistake.)

|      	| Predicted: Benign 	| Predicted: Malignant 	|
|:----:	|----	|----	|
| Actual: Benign 	|    7000	|   3100 	|
| Actual: Malignant  	|    30	|   180 	|

#### Answer to question 4 (b)

Write your answer here.